<a href="https://colab.research.google.com/github/sindhu-2507/Speech_Emotion_Recognition/blob/master/Speech_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sindhupawar","key":"175fbdf68e584cf19b056157b9c4f2af"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

 97% 417M/429M [00:11<00:00, 31.9MB/s]
100% 429M/429M [00:11<00:00, 37.9MB/s]


In [5]:
!mkdir data
%cd data/
!unzip /content/ravdess-emotional-speech-audio.zip

/content/data
Archive:  /content/ravdess-emotional-speech-audio.zip
  inflating: Actor_01/03-01-01-01-01-01-01.wav  
  inflating: Actor_01/03-01-01-01-01-02-01.wav  
  inflating: Actor_01/03-01-01-01-02-01-01.wav  
  inflating: Actor_01/03-01-01-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-01-01-01-01.wav  
  inflating: Actor_01/03-01-02-01-01-02-01.wav  
  inflating: Actor_01/03-01-02-01-02-01-01.wav  
  inflating: Actor_01/03-01-02-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-02-01-01-01.wav  
  inflating: Actor_01/03-01-02-02-01-02-01.wav  
  inflating: Actor_01/03-01-02-02-02-01-01.wav  
  inflating: Actor_01/03-01-02-02-02-02-01.wav  
  inflating: Actor_01/03-01-03-01-01-01-01.wav  
  inflating: Actor_01/03-01-03-01-01-02-01.wav  
  inflating: Actor_01/03-01-03-01-02-01-01.wav  
  inflating: Actor_01/03-01-03-01-02-02-01.wav  
  inflating: Actor_01/03-01-03-02-01-01-01.wav  
  inflating: Actor_01/03-01-03-02-01-02-01.wav  
  inflating: Actor_01/03-01-03-02-02-01-01.wav  
 

In [6]:
import os
import random
import sys
dir_path = '/content/data'
dir_list=os.listdir(dir_path)
dir_list.sort()
print (dir_list)


['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24', 'audio_speech_actors_01-24']


In [7]:
dir_list = dir_list[0:24]
print (dir_list)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [8]:
# %cd /content
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [9]:
import librosa

In [10]:
!pip install --upgrade keras

  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [29]:
#All library imports
%matplotlib inline
import csv
import random
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
# import tensorflow as tf
#This is simply a linear stack of neural network layers, and it's perfect for the type of feed-forward CNN
from keras.models import Sequential, load_model
#keras core layers
from keras.layers import Dense, Dropout, Activation, Flatten
#These are the convolutional layers that will help us efficiently train on the data
from keras.layers import Convolution2D, MaxPooling2D
#Utilities
from keras.utils import np_utils

In [12]:
# Extracting the features
def extract_features(directory_path,directory_list):
  X = []
  Y = []
  w = 0
  curr_dir = directory_path
  dir_list = directory_list
  for i in dir_list:
    dir_path = os.path.join(curr_dir,i)
    for j in os.listdir(dir_path):
      print(i)
      features = []
      songname = os.path.join(dir_path,j)
      p = j.split("-")
      y, sr = librosa.load(songname)
      mfcc = (librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40))
      X.append(mfcc)
      Y.append(int(p[2])-1)
      q = len(mfcc[0])
      w = max(w,q)
  return X,Y,w

In [14]:
path = "/content/data"
X,Y,w = extract_features(path,dir_list)

Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_01
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
Actor_02
A

In [15]:
# Preprocessing the data
def preprocess(X,Y,w):
  x = np.empty([1440,40,228])
  for i in range(len(X)):
    # print(len(X[0][0]))
    for j in range(len(X[i])):
      q = np.asarray(X[i][j])
      N = w - len(q)
      s = np.pad(q,(0,N),'constant')
      x[i][j] = s
  X = np.asarray(x)
  Y = np.asarray(Y)
  return X,Y

In [16]:
X,Y = preprocess(X,Y,w)
print(X.shape)
print(Y.shape)

(1440, 40, 228)
(1440,)


In [17]:
#Reshaping the X and Y data
def reshape(X):
# print(X.shape)
  X = np.reshape(X,[1440,1,40,228])
  # print(X.shape)
  # print(X[0])
  return X

In [18]:
X = reshape(X)

In [19]:
#dividing the data into train,dev and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
Y_train = np_utils.to_categorical(y_train,8)
Y_test = np_utils.to_categorical(y_test,8)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1296, 1, 40, 228)
(1296, 8)
(144, 1, 40, 228)
(144, 8)


In [20]:
#initialising the model
model = Sequential()

In [21]:
# best fit till now
model.add(Convolution2D(64, 3 , 3, input_shape=(1,40,228), data_format = 'channels_first'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Convolution2D(64,3,3))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [23]:
model.fit(X_train, Y_train, batch_size=36, epochs=76)

Epoch 1/76
36/36 [==============================] - 0s 7ms/step - loss: 7.5032 - accuracy: 0.1489
Epoch 2/76
36/36 [==============================] - 0s 7ms/step - loss: 2.0283 - accuracy: 0.1991
Epoch 3/76
36/36 [==============================] - 0s 7ms/step - loss: 1.9703 - accuracy: 0.2184
Epoch 4/76
36/36 [==============================] - 0s 7ms/step - loss: 1.8800 - accuracy: 0.2485
Epoch 5/76
36/36 [==============================] - 0s 7ms/step - loss: 1.8375 - accuracy: 0.2801
Epoch 6/76
36/36 [==============================] - 0s 7ms/step - loss: 1.7896 - accuracy: 0.3009
Epoch 7/76
36/36 [==============================] - 0s 7ms/step - loss: 1.7771 - accuracy: 0.3133
Epoch 8/76
36/36 [==============================] - 0s 7ms/step - loss: 1.6884 - accuracy: 0.3341
Epoch 9/76
36/36 [==============================] - 0s 7ms/step - loss: 1.7190 - accuracy: 0.3194
Epoch 10/76
36/36 [==============================] - 0s 7ms/step - loss: 1.6703 - accuracy: 0.3472
Epoch 11/76
36/36 [

In [24]:
score = model.evaluate(X_test,Y_test, verbose = 0)
y_pred = model.predict_classes(X_test)
print(y_test)
print(y_pred)
print(score)
wrong = 0
for i in range(len(y_test)):
  if(y_test[i]!=y_pred[i]):
    wrong = wrong+1
print(wrong)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[2 6 4 3 5 6 1 3 3 2 7 4 2 3 5 5 6 5 3 7 5 4 1 4 2 3 0 1 3 4 5 3 2 3 3 5 2
 0 1 6 7 0 0 6 3 4 2 3 0 7 2 4 6 4 5 1 2 0 2 6 3 4 6 3 3 2 3 2 6 6 0 5 1 7
 2 4 1 0 3 5 5 3 7 7 7 6 0 6 5 3 5 2 4 3 2 5 3 4 7 2 4 5 4 3 7 1 3 5 7 2 6
 1 6 0 2 7 1 6 5 3 0 7 2 3 2 0 5 2 3 5 5 3 2 3 7 3 7 6 1 0 6 6 7 1]
[7 5 4 3 5 6 1 1 3 2 7 2 5 3 5 5 5 5 3 7 5 4 1 2 0 1 0 1 3 6 5 3 2 2 3 5 2
 0 1 6 7 0 0 6 1 4 2 3 0 5 2 4 6 4 5 3 2 0 2 3 3 4 6 3 5 2 1 2 1 0 2 3 1 7
 2 4 1 2 3 7 6 1 1 5 6 2 1 1 3 0 2 4 4 5 2 5 3 4 5 2 4 5 4 3 5 1 3 5 1 4 6
 3 3 0 3 7 1 1 5 3 2 7 7 2 7 1 2 4 3 5 5 1 2 5 4 1 7 6 1 0 6 6 1 1]
[2.755282402038574, 0.6111111044883728]
56


In [25]:
y_pred_train = model.predict_classes(X_train)
print(y_pred_train)
print(y_train)

[5 4 0 ... 7 4 1]
[5 4 0 ... 7 4 1]


In [26]:
wrong = 0
for i in range(len(y_train)):
  if(y_train[i]!=y_pred_train[i]):
    wrong = wrong + 1
print(wrong)

1


In [ ]:
upload = files.upload()

Saving happy.wav to happy (1).wav


In [ ]:
os.path.exists("/content/data/angry.wav")

True

In [ ]:
my_X = []
my_Y = []
t = 0
# songname = "/content/data/angry.wav"
# y, sr = librosa.load(songname)
# mfcc = (librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40))
# my_X.append(mfcc)
# i = len(mfcc[0])
# t = max(t, i)
# my_Y.append(4)
songname = "/content/data/happy.wav"
y, sr = librosa.load(songname)
mfcc = (librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40))
my_X.append(mfcc)
i = len(mfcc[0])
t = max(t, i)
my_Y.append(2)
# songname = "/content/data/neutral.wav"
# y, sr = librosa.load(songname)
# mfcc = (librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40))
# my_X.append(mfcc)
# i = len(mfcc[0])
# t = max(t, i)
# my_Y.append(0)

In [ ]:
print(i)
print(len(my_X),len(my_X[0]),len(my_X[0][0]))

128
1 40 128


In [ ]:
my_x = np.empty([1,40,228])
for i in range(len(my_X)):
  # print(len(X[0][0]))
  for j in range(len(my_X[i])):
    q = np.asarray(my_X[i][j])
    N = w - len(q)
    s = np.pad(q,(0,N),'constant')
    my_x[i][j] = s
my_X = np.asarray(my_x)
my_Y = np.asarray(my_Y)
print(my_X.shape)
print(my_Y.shape)

(1, 40, 228)
(1,)


In [ ]:
my_X = np.reshape(my_X,[1,1,40,228])
my_Y = my_Y
print(my_X.shape)
print(my_Y)

(1, 1, 40, 228)
[2]


In [ ]:
my_y_pred = model.predict_classes(my_X)

In [ ]:
print(my_y_pred)

[2]
